## Remove Betting Data

Since around the middle of the 2020 season, `footywire.com` hasn't been posting betting data, which means one of my stronger features is now blank, which can't be good for model performance. I suspect that removing it will result in a loss in performance, but it can't be any worse than trying to predict without a bunch of data that was present at training time.

## Code Setup

In [1]:
%load_ext autoreload

/usr/local/lib/python3.8/site-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [2]:
%autoreload 2

import numpy as np
import pandas as pd
from sklearn.metrics import mean_absolute_error


from augury.ml_estimators import StackingEstimator
from augury.sklearn.metrics import match_accuracy_scorer
from augury.ml_data import MLData
from augury.settings import CV_YEAR_RANGE, SEED
from augury.model_tracking import score_model

np.random.seed(SEED)

/usr/local/lib/python3.8/site-packages/patsy/constraint.py:13: DeprecationWarning: Using or importing the ABCs from 'collections' instead of from 'collections.abc' is deprecated since Python 3.3, and in 3.9 it will stop working
  from collections import Mapping


2021-02-10 09:27:42,080 - matplotlib.font_manager - INFO - Generating new fontManager, this may take some time...


In [3]:
data = MLData(train_year_range=(max(CV_YEAR_RANGE),))
data.data

/usr/local/lib/python3.8/site-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


2021-02-10 09:27:43,369 - kedro.io.data_catalog - INFO - Loading data from `full_data` (JSONDataSet)...


team      oppo_team round_type  \
Adelaide         1991 1           Adelaide       Hawthorn    Regular   
                      2           Adelaide        Carlton    Regular   
                      3           Adelaide         Sydney    Regular   
                      4           Adelaide       Essendon    Regular   
                      5           Adelaide     West Coast    Regular   
...                                    ...            ...        ...   
Western Bulldogs 2021 19  Western Bulldogs       Adelaide    Regular   
                      20  Western Bulldogs      Melbourne    Regular   
                      21  Western Bulldogs       Essendon    Regular   
                      22  Western Bulldogs       Hawthorn    Regular   
                      23  Western Bulldogs  Port Adelaide    Regular   

                                   venue prev_match_oppo_team  \
Adelaide         1991 1    Football Park                    0   
                      2    Football Park             Hawthorn   
                      3           S.C.G.              Carlton   
                      4       Windy Hill               Sydney   
                      5          Subiaco             Essendon   
...                                  ...                  ...   
Western Bulldogs 2021 19  Eureka Stadium           Gold Coast   
                      20          M.C.G.             Adelaide   
                      21       Docklands            Melbourne   
                      22       York Park             Essendon   
                      23       Docklands             Hawthorn   

                         oppo_prev_match_oppo_team                      date  \
Adelaide         1991 1                  Melbourne 1991-03-22 03:56:00+00:00   
                      2                    Fitzroy 1991-03-31 03:56:00+00:00   
                      3                   Hawthorn 1991-04-07 03:05:00+00:00   
                      4            North Melbourne 1991-04-13 03:30:00+00:00   
                      5            North Melbourne 1991-04-21 05:27:00+00:00   
...                                            ...                       ...   
Western Bulldogs 2021 19                West Coast 2021-07-24 02:20:00+00:00   
                      20                Gold Coast 2021-07-31 02:20:00+00:00   
                      21                    Sydney 2021-08-07 02:20:00+00:00   
                      22               Collingwood 2021-08-14 02:11:00+00:00   
                      23                   Carlton 2021-08-21 02:20:00+00:00   

                          team_goals  team_behinds  score  ...  \
Adelaide         1991 1           24            11    155  ...   
                      2           12             9     81  ...   
                      3           19            18    132  ...   
                      4            6            11     47  ...   
                      5            9            11     65  ...   
...                              ...           ...    ...  ...   
Western Bulldogs 2021 19           0             0      0  ...   
                      20           0             0      0  ...   
                      21           0             0      0  ...   
                      22           0             0      0  ...   
                      23           0             0      0  ...   

                          oppo_rolling_prev_match_time_on_ground_skew  \
Adelaide         1991 1                                           0.0   
                      2                                           0.0   
                      3                                           0.0   
                      4                                           0.0   
                      5                                           0.0   
...                                                               ...   
Western Bulldogs 2021 19                                          0.0   
                      20                              

In [4]:
X_train, y_train = data.train_data
X_test, y_test = data.test_data

/usr/local/lib/python3.8/site-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [5]:
legacy_data = MLData(train_year_range=(max(CV_YEAR_RANGE),), data_set='legacy_data')
legacy_data.data

2021-02-10 09:28:31,280 - kedro.io.data_catalog - INFO - Loading data from `legacy_data` (JSONDataSet)...


team      oppo_team round_type  \
Adelaide         1991 1           Adelaide       Hawthorn    Regular   
                      2           Adelaide        Carlton    Regular   
                      3           Adelaide         Sydney    Regular   
                      4           Adelaide       Essendon    Regular   
                      5           Adelaide     West Coast    Regular   
...                                    ...            ...        ...   
Western Bulldogs 2021 19  Western Bulldogs       Adelaide    Regular   
                      20  Western Bulldogs      Melbourne    Regular   
                      21  Western Bulldogs       Essendon    Regular   
                      22  Western Bulldogs       Hawthorn    Regular   
                      23  Western Bulldogs  Port Adelaide    Regular   

                                   venue prev_match_oppo_team  \
Adelaide         1991 1    Football Park                    0   
                      2    Football Park             Hawthorn   
                      3           S.C.G.              Carlton   
                      4       Windy Hill               Sydney   
                      5          Subiaco             Essendon   
...                                  ...                  ...   
Western Bulldogs 2021 19  Eureka Stadium           Gold Coast   
                      20          M.C.G.             Adelaide   
                      21       Docklands            Melbourne   
                      22       York Park             Essendon   
                      23       Docklands             Hawthorn   

                         oppo_prev_match_oppo_team                      date  \
Adelaide         1991 1                  Melbourne 1991-03-22 03:56:00+00:00   
                      2                    Fitzroy 1991-03-31 03:56:00+00:00   
                      3                   Hawthorn 1991-04-07 03:05:00+00:00   
                      4            North Melbourne 1991-04-13 03:30:00+00:00   
                      5            North Melbourne 1991-04-21 05:27:00+00:00   
...                                            ...                       ...   
Western Bulldogs 2021 19                West Coast 2021-07-24 02:20:00+00:00   
                      20                Gold Coast 2021-07-31 02:20:00+00:00   
                      21                    Sydney 2021-08-07 02:20:00+00:00   
                      22               Collingwood 2021-08-14 02:11:00+00:00   
                      23                   Carlton 2021-08-21 02:20:00+00:00   

                          team_goals  team_behinds  score  ...  \
Adelaide         1991 1           24            11    155  ...   
                      2           12             9     81  ...   
                      3           19            18    132  ...   
                      4            6            11     47  ...   
                      5            9            11     65  ...   
...                              ...           ...    ...  ...   
Western Bulldogs 2021 19           0             0      0  ...   
                      20           0             0      0  ...   
                      21           0             0      0  ...   
                      22           0             0      0  ...   
                      23           0             0      0  ...   

                          oppo_rolling_prev_match_goals_divided_by_rolling_prev_match_goals_plus_rolling_prev_match_behinds  \
Adelaide         1991 1                                                   0                                                   
                      2                                                   0                                                   
                      3                                                   0                                                   
                      4                                                   0                                                   
   

In [6]:
X_train_legacy, y_train_legacy = legacy_data.train_data
X_test_legacy, y_test_legacy = legacy_data.test_data

/usr/local/lib/python3.8/site-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


## Check baseline model performance

Default models and data sets have been updated to omit betting data. Legacy data still has betting data.

### Stacking model with legacy data

In [10]:
stacking_estimator = StackingEstimator()

In [12]:
legacy_stacking_estimator_scores = score_model(stacking_estimator, legacy_data, n_jobs=-1)

legacy_stacking_estimator_scores

/usr/local/lib/python3.8/site-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)
[Parallel(n_jobs=-1)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=-1)]: Done   5 out of   5 | elapsed:  3.0min finished


{'fit_time': array([ 98.42565656, 106.00997615, 106.62583876, 112.20423508,
         80.24857163]),
 'score_time': array([1.40320921, 1.59928989, 1.3297472 , 1.17869186, 1.11199951]),
 'test_neg_mean_absolute_error': array([-30.1726335 , -27.76617796, -27.90377486, -25.45517358,
        -27.29013594]),
 'test_match_accuracy': array([0.73300971, 0.72463768, 0.69565217, 0.73429952, 0.68115942])}

In [13]:
print('Mean accuracy:', legacy_stacking_estimator_scores['test_match_accuracy'].mean())
print('Mean MAE:', abs(legacy_stacking_estimator_scores['test_neg_mean_absolute_error'].mean()))

/usr/local/lib/python3.8/site-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


Mean accuracy: 0.7137517002016791
Mean MAE: 27.717579165044178


### Stacking model with updated data (no betting data)

In [14]:
stacking_estimator_scores = score_model(stacking_estimator, data, n_jobs=-1)

stacking_estimator_scores

[Parallel(n_jobs=-1)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=-1)]: Done   5 out of   5 | elapsed:  3.0min finished


{'fit_time': array([ 90.97952986,  93.43376255,  95.69725657, 102.91990948,
         73.64427876]),
 'score_time': array([1.41234684, 1.79324412, 1.14672923, 1.21438718, 1.16238284]),
 'test_neg_mean_absolute_error': array([-30.31578799, -29.32395939, -27.90862684, -26.15584625,
        -28.03885799]),
 'test_match_accuracy': array([0.70873786, 0.73429952, 0.66666667, 0.73429952, 0.63768116])}

In [15]:
print('Mean accuracy:', stacking_estimator_scores['test_match_accuracy'].mean())
print('Mean MAE:', abs(stacking_estimator_scores['test_neg_mean_absolute_error'].mean()))

/usr/local/lib/python3.8/site-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


Mean accuracy: 0.6963369447962103
Mean MAE: 28.34861569224795


In [16]:
print(
    'Mean accuracy lost:',
    legacy_stacking_estimator_scores['test_match_accuracy'].mean() - stacking_estimator_scores['test_match_accuracy'].mean()
)
print(
    'Mean MAE lost:',
    abs(stacking_estimator_scores['test_neg_mean_absolute_error'].mean()) - abs(legacy_stacking_estimator_scores['test_neg_mean_absolute_error'].mean())
)

Mean accuracy lost: 0.017414755405468774
Mean MAE lost: 0.6310365272037721


## Conclusion

As expected, there was a small loss in performance, but given that the default parameters are currently tuned for a model that has betting data features, and the fact that we won't have betting data in the future, a model optimised to do without betting data will likely perform better this season. At the very least, a loss of 1.7% in accuracy certainly isn't worth maintaining a Selenium-based web scraper just to get updated betting data every week.